# Entity_soft_voting

## 필수코드

In [ ]:
# 필요 시 설치(가상환경의 경우 터미널에 설치)
!pip install transformers==4.24.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 13.5 MB/s 
     |████████████████████████████████| 7.6 MB 51.6 MB/s 
     |████████████████████████████████| 163 kB 58.9 MB/s 


In [ ]:
# 필요 시 설치
!pip install datasets==2.6.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 15.4 MB/s 
     |████████████████████████████████| 115 kB 96.2 MB/s 
     |████████████████████████████████| 212 kB 94.4 MB/s 
     |████████████████████████████████| 95 kB 962 kB/s 
     |████████████████████████████████| 127 kB 78.6 MB/s 
     |████████████████████████████████| 115 kB 75.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: dill
    Found existing installation: dill 0.3.6
    Uninstalling dill-0.3.6:
      Successfully uninstalled dill-0.3.6


모듈 import

In [ ]:
import json
import os
import torch
import torch.nn as nn
from tqdm import trange
from transformers import AutoModel, AutoTokenizer
from torch.utils.data import DataLoader, TensorDataset
from transformers import get_linear_schedule_with_warmup
from transformers import AdamW
from datasets import load_metric
from sklearn.metrics import f1_score
import pandas as pd
import copy

## 전역 변수 설정

In [ ]:
# 최대 단어 수 설정
max_len = 256
batch_size = 8
# soft_voting시 사용할 베이스 모델 설정
base_model = 'kykim/electra-kor-base'
learning_rate = 3e-6
eps = 1e-8
num_train_epochs = 20
classifier_hidden_size = 768
classifier_dropout_prob = 0.1

# 개체#속성 쌍 설정(label25)
entity_property_pair = [
    '제품 전체#일반', '제품 전체#가격', '제품 전체#디자인', '제품 전체#품질', '제품 전체#편의성', '제품 전체#인지도', '제품 전체#다양성',
    '본품#일반', '본품#디자인', '본품#품질', '본품#편의성', '본품#다양성', '본품#가격', '본품#인지도',
    '패키지/구성품#일반', '패키지/구성품#디자인', '패키지/구성품#품질', '패키지/구성품#편의성', '패키지/구성품#가격', '패키지/구성품#다양성',
    '브랜드#일반', '브랜드#가격', '브랜드#품질', '브랜드#인지도', '브랜드#디자인'
                    ]

# 개체#속성 쌍 설정(label23)
# entity_property_pair = [
#     '제품 전체#일반', '제품 전체#가격', '제품 전체#디자인', '제품 전체#품질', '제품 전체#편의성', '제품 전체#인지도',
#     '본품#일반', '본품#디자인', '본품#품질', '본품#편의성', '본품#다양성', '본품#가격', '본품#인지도',
#     '패키지/구성품#일반', '패키지/구성품#디자인', '패키지/구성품#품질', '패키지/구성품#편의성', '패키지/구성품#가격', '패키지/구성품#다양성',
#     '브랜드#일반', '브랜드#가격', '브랜드#품질', '브랜드#인지도',
#                     ]

# 문장과 개체#속성 쌍의 관계를 True, False 로 표시
tf_id_to_name = ['True', 'False']
tf_name_to_id = {tf_id_to_name[i]: i for i in range(len(tf_id_to_name))}

# 문장과 개체#속성 쌍의 관계로 감성을 positive, negative, neutral 로 표시
polarity_id_to_name = ['positive', 'negative', 'neutral']
polarity_name_to_id = {polarity_id_to_name[i]: i for i in range(len(polarity_id_to_name))}

# 그래픽 카드 사용 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 불러온 tokenizer 에 special token 을 추가
special_tokens_dict = {
    'additional_special_tokens': ['&name&', '&affiliation&', '&social-security-num&', '&tel-num&', '&card-num&', '&bank-account&', '&num&', '&online-account&']
}

## json 파일 로드

In [ ]:
# json 파일 읽어서 list에 저장
def jsonload(fname, encoding="utf-8"):
    with open(fname, encoding=encoding) as f:
        j = json.load(f)
    return j

# json 개체를 파일이름으로 저장
def jsondump(j, fname):
    with open(fname, "w", encoding="UTF8") as f:
        json.dump(j, f, ensure_ascii=False)

# jsonl 파일 읽어서 list에 저장
def jsonlload(fname, encoding="utf-8"):
    json_list = []
    with open(fname, encoding=encoding) as f:
        for line in f.readlines():
            json_list.append(json.loads(line))
    return json_list

## 모델 정의

### electra 모델을 기반으로 한 classification 모델 이용

In [ ]:
# 아래 ELECTRABaseClassifier의 classifier로 사용될 class
class SimpleClassifier(nn.Module):

    def __init__(self, num_label):
        super().__init__()
        self.dense = nn.Linear(classifier_hidden_size, classifier_hidden_size)
        self.dropout = nn.Dropout(classifier_dropout_prob)
        self.output = nn.Linear(classifier_hidden_size, num_label)

    def forward(self, features):
        x = features[:, 0, :]
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.output(x)
        return x

# 불러올 base model 기반 classification 설정
class ELECTRABaseClassifier(nn.Module):
    def __init__(self, num_label, len_tokenizer):
        super(ELECTRABaseClassifier, self).__init__()

        self.num_label = num_label
        self.xlm_roberta = AutoModel.from_pretrained(base_model)
        self.xlm_roberta.resize_token_embeddings(len_tokenizer)

        self.labels_classifier = SimpleClassifier(self.num_label)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.xlm_roberta(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=None
        )

        sequence_output = outputs[0]
        logits = self.labels_classifier(sequence_output)

        loss = None

        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_label),
                                                labels.view(-1))

        return loss, logits

## 로드된 모델 값으로 soft_voting

In [ ]:
# 인자로 받아온 모델들을 통해 predict 하는 문장 데이터에 label(entity_property, polarity) 해주는 함수
#                                       모델 갯수에 따라서 ce_model 추가 또는 삭제
def predict_from_korean_form(tokenizer, ce_model1, ce_model2, ce_model3, ce_model4, ce_model5, pc_model, data):

    # 모델 갯수에 따라서 ce_model 추가 또는 삭제
    ce_model1.to(device)
    ce_model1.eval()

    ce_model2.to(device)
    ce_model2.eval()

    ce_model3.to(device)
    ce_model3.eval()

    ce_model4.to(device)
    ce_model4.eval()

    ce_model5.to(device)
    ce_model5.eval()

    for sentence in data:
        form = sentence['sentence_form']
        sentence['annotation'] = []
        if type(form) != str:
            print("form type is arong: ", form)
            continue

        # 개체#속성 쌍을 순서대로 입력
        for pair in entity_property_pair:
            # 문장과 pair를 tokenizer화 
            tokenized_data = tokenizer(form, pair, padding='max_length', max_length=256, truncation=True)

            # tonkenizer 된 값으로 entity_property 모델을 통해 True, False 결과값을 출력
            input_ids = torch.tensor([tokenized_data['input_ids']]).to(device)
            attention_mask = torch.tensor([tokenized_data['attention_mask']]).to(device)

            # 문장에 대한 각 모델별 점수 예측 결과 저장
            with torch.no_grad():
                # 모델 갯수에 따라 추가 또는 삭제
                _, ce_logits1 = ce_model1(input_ids, attention_mask)
                _, ce_logits2 = ce_model2(input_ids, attention_mask)
                _, ce_logits3 = ce_model3(input_ids, attention_mask)
                _, ce_logits4 = ce_model4(input_ids, attention_mask)
                _, ce_logits5 = ce_model5(input_ids, attention_mask)

            # 각 모델별 예측한 결과값을 합산하여 평균 구하기
              # 모델 갯수에 따라 추가 또는 삭제
            logits1 = (ce_logits1[0][0]+ce_logits2[0][0]+ce_logits3[0][0]+ce_logits4[0][0]+ce_logits5[0][0])/5
            logits2 = (ce_logits1[0][1]+ce_logits2[0][1]+ce_logits3[0][1]+ce_logits4[0][1]+ce_logits5[0][1])/5

            ce_logits = torch.tensor([[logits1, logits2]]).to(device)

            ce_predictions = torch.argmax(ce_logits, dim = -1)

            ce_result = tf_id_to_name[ce_predictions[0]]

            # True 일 경우에만 다음 코드 진행
            if ce_result == 'True':

                # tonkenizer 된 값으로 polarity 모델을 통해 Positive, Negative,Neutral 결과값을 출력                
                with torch.no_grad():
                    _, pc_logits = pc_model(input_ids, attention_mask)

                pc_predictions = torch.argmax(pc_logits, dim=-1)
                pc_result = polarity_id_to_name[pc_predictions[0]]

                # entity_property 모델과 polarity 모델을 통해서 출력 된 결과 값을 label 입력
                sentence['annotation'].append([pair, pc_result])

    return data

## 지정된 모델들 불러오기

In [ ]:
# 불러올 entity_property pt 파일 경로 설정
# 5개인 경우 예시
entity_classification_model_path = [
    ['./',
     './',
     './',
     './',
     './']
]

# 불러올 polarity pt 파일 경로 설정
polarity_classification_model_path = './'

# 학습 후 저장된 pt 파일로 predict 할 json 데이터 설정
test_data_path = './'

In [ ]:
# 변수로 지정된 경로에 따라 predict 파일에 labeling 후 저장하는 함수
def test_sentiment_analysis():

    # predict 시 사용할 tokenizer와 데이터를 불러오기
    tokenizer = AutoTokenizer.from_pretrained(base_model)
    num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
    test_data = jsonlload(test_data_path)
    
    # entity_property 모델 불러오기
    # 모델 갯수에 따라 model1 추가 또는 삭제
    model1 = ELECTRABaseClassifier(len(tf_id_to_name), len(tokenizer))
    model1.load_state_dict(torch.load(entity_classification_model_path[0][0], map_location=device))
    model1.to(device)
    model1.eval()

    # 모델 갯수에 따라 model2추가 또는 삭제
    model2 = ELECTRABaseClassifier(len(tf_id_to_name), len(tokenizer))
    model2.load_state_dict(torch.load(entity_classification_model_path[0][1], map_location=device))
    model2.to(device)
    model2.eval()

    # 모델 갯수에 따라 model3 추가 또는 삭제
    model3 = ELECTRABaseClassifier(len(tf_id_to_name), len(tokenizer))
    model3.load_state_dict(torch.load(entity_classification_model_path[0][2], map_location=device))
    model3.to(device)
    model3.eval()

    # 모델 갯수에 따라 model4 추가 또는 삭제
    model4 = ELECTRABaseClassifier(len(tf_id_to_name), len(tokenizer))
    model4.load_state_dict(torch.load(entity_classification_model_path[0][3], map_location=device))
    model4.to(device)
    model4.eval()

    # 모델 갯수에 따라 model5 추가 또는 삭제
    model5 = ELECTRABaseClassifier(len(tf_id_to_name), len(tokenizer))
    model5.load_state_dict(torch.load(entity_classification_model_path[0][4], map_location=device))
    model5.to(device)
    model5.eval()
            
    # polarity 모델 불러오기   
    polarity_model = ELECTRABaseClassifier(len(polarity_id_to_name), len(tokenizer))
    polarity_model.load_state_dict(torch.load(polarity_classification_model_path, map_location=device))
    polarity_model.to(device)
    polarity_model.eval()
    
    # predict_from_korean_form 함수를 통해 predict 데이터 만들기
    # 모델 갯수에 따라 model 추가 또는 삭제
    pred_data = predict_from_korean_form(tokenizer, model1, model2, model3, model4, model5, model6, model7, model8, model9, model10, polarity_model, copy.deepcopy(test_data))

    # 만들어진 predict 데이터를 json 파일로 저장
    jsondump(pred_data, './')

## 실행코드

In [ ]:
# 변수로 지정된 경로에 따라 predict 파일을 생성 및 저장 함수
test_sentiment_analysis()

Downloading:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/620 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/344k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/473M [00:00<?, ?B/s]

Some weights of the model checkpoint at kykim/electra-kor-base were not used when initializing ElectraModel: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at kykim/electra-kor-base were not used when initializing ElectraModel: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_predict